In [1]:
# !python3 -mpip install scikit-learn

In [2]:
from __future__ import print_function, absolute_import
import _init_paths
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import argparse
from Metrics import *
import torch
import numpy as np
from train_utils import *
# from Data2array import *
from pose_code.hourglass import hg
from pose_code.atthourglass import atthg
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
import skimage
import pickle
import colorsys
from torch.utils.data import DataLoader
from sklearn.utils.extmath import cartesian
%matplotlib inline

In [3]:
from torchvision.utils import make_grid
def save_test_results(inputs, outputs, targets, name='', target_th=0.5):
    clr_vis_Y = []
    hues = np.linspace(0, 179, targets.shape[1], dtype=np.uint8)
    blank_ch = 255*np.ones_like(targets[0,0], dtype=np.uint8)

    for Y in [targets, outputs]:
        for y, x in zip(Y, inputs):
            y_colored = np.zeros([y.shape[1], y.shape[2], 3], dtype=np.uint8)
            y_all = np.zeros([y.shape[1], y.shape[2]], dtype=np.uint8)
            
            for ych, hue_i in zip(y, hues):
                ych = ych/np.max(np.max(ych))
                ych[np.where(ych<target_th)] = 0
                # ych = cv2.GaussianBlur(ych,(15,15),cv2.BORDER_DEFAULT)

                ych_hue = np.ones_like(ych, dtype=np.uint8)*hue_i
                ych = np.uint8(255*ych/np.max(ych))

                colored_ych = np.zeros_like(y_colored, dtype=np.uint8)
                colored_ych[:, :, 0] = ych_hue
                colored_ych[:, :, 1] = blank_ch
                colored_ych[:, :, 2] = ych
                colored_y = cv2.cvtColor(colored_ych, cv2.COLOR_HSV2BGR)

                y_colored += colored_y
                y_all += ych

            x = np.moveaxis(x, 0, -1)
            x = x/np.max(x)*255

            x_3ch = np.zeros([x.shape[0], x.shape[1], 3])
            for i in range(3):
                x_3ch[:, :, i] = x[:, :, 0]

            img_mix = np.uint8(x_3ch*0.5 + y_colored*0.5)
            # img_mix = cv2.cvtColor(img_mix, cv2.COLOR_BGR2RGB)
            clr_vis_Y.append(img_mix)

    
    t = np.array(clr_vis_Y)
    t = np.transpose(t, [0, 3, 1, 2])
    trgts = make_grid(torch.Tensor(t), nrow=4)

    txt = f'../visualize/{name}_test_result.png'
    res = np.transpose(trgts.numpy(), (1,2,0))
    cv2.imwrite(txt, res)

In [4]:
parser = argparse.ArgumentParser(description='Verterbal disc labeling using pose estimation')
    
## Parameters
datapathds = '../prepared_dataset/prepared_testset_t1_ds'
datapathfull='../prepared_dataset/prepared_testset_t1_full'              
modality='t1'                                               
njoints=11
resume= False
att= True
stacks=2
blocks=1

In [5]:
import cv2
idtest = 1
def extract_skeleton(inputs, outputs, target, target_th=0.4):
    global idtest
    outputs  = outputs.data.cpu().numpy()
    target  = target.data.cpu().numpy()
    inputs = inputs.data.cpu().numpy()
    skeleton_images = []
    for idx in range(outputs.shape[0]):    
        count_list = []
        Nch = 0
        center_list = {}
        while np.sum(np.sum(target[idx, Nch]))>0:
              Nch += 1       
        for idy in range(Nch): 
            ych = outputs[idx, idy]
            ych = np.rot90(ych)
            ych = ych/np.max(np.max(ych))
            ych[np.where(ych<target_th)] = 0
            ych = np.where(ych>0, 1.0, 0)
            ych = np.uint8(ych)
            num_labels, labels_im, states, centers = cv2.connectedComponentsWithStats(ych)
            count_list.append(num_labels-1)
            center_list[str(idy)] = [t[::-1] for t in centers[1:]]
            
        ups = []
        for c in count_list:
            ups.append(range(c))
        combs = cartesian(ups)
        
        best_loss = np.Inf
        best_skeleton = []
        for comb in combs:
            cnd_skeleton = []
            for joint_idx, cnd_joint_idx in enumerate(comb):
                cnd_center = center_list[str(joint_idx)][cnd_joint_idx]
                cnd_skeleton.append(cnd_center)
            loss = check_skeleton(cnd_skeleton, norm_mean_skeleton)
            if best_loss > loss:
                best_loss = loss
                best_skeleton = cnd_skeleton
        
        hits = np.zeros_like(outputs[0])
        for i, jp, in enumerate(best_skeleton):
            jp = [int(t) for t in jp]
            hits[i, jp[0]-1:jp[0]+2, jp[1]-1:jp[1]+2] = [255, 255, 255]
            hits[i, :, :] = cv2.GaussianBlur(hits[i, :, :],(5,5),cv2.BORDER_DEFAULT)
            hits[i, :, :] = hits[i, :, :]/hits[i, :, :].max()*255
        
        skeleton_images.append(hits)
        
    skeleton_images = np.array(skeleton_images)
    inputs = np.rot90(inputs, axes=(-2, -1))
    target = np.rot90(target, axes=(-2, -1))
    save_test_results(inputs, skeleton_images, targets=target, name=idtest, target_th=0.5)
    idtest+=1
    return skeleton_images

In [6]:
def check_skeleton(cnd_sk, mean_skeleton):
    cnd_sk = np.array(cnd_sk)
    Normjoint = np.linalg.norm(cnd_sk[0]-cnd_sk[4])
    for idx in range(1, len(cnd_sk)):
        cnd_sk[idx] = (cnd_sk[idx] - cnd_sk[0]) / Normjoint
    cnd_sk[0] -= cnd_sk[0]
    
    return np.sum(np.linalg.norm(mean_skeleton[:len(cnd_sk)]-cnd_sk))
    

In [7]:
# main script

args = parser.parse_known_args()
norm_mean_skeleton = np.load('../prepared_dataset/T1_Skelet.npy')

global cuda_available
cuda_available = torch.cuda.is_available()
print('load image')

with open(datapathds, 'rb') as file_pi:
     ds = pickle.load(file_pi)
with open(datapathfull, 'rb') as file_pi:
     full = pickle.load(file_pi)            
full[0] = full[0][:, :, :, :, 0]
print('retrieving ground truth coordinates')

# coord_gt = retrieves_gt_coord(ds)
# intialize metrics
global distance_l2
global zdis
global faux_pos
global faux_neg
global tot
distance_l2 = []
zdis = []
faux_pos = []
faux_neg = []
tot = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if att:
    model = atthg(num_stacks=stacks, num_blocks=blocks, num_classes=njoints)
    model = torch.nn.DataParallel(model).to(device)
    model.load_state_dict(torch.load(f'../weights/model_{modality}_att', map_location='cpu')['model_weights'])
else:
    model = hg(num_stacks=stacks, num_blocks=blocks, num_classes=njoints)
    model = torch.nn.DataParallel(model).to(device)
    model.load_state_dict(torch.load(f'../weights/model_simple_{modality}_att', map_location='cpu')['model_weights'])


## Get the visualization resutls of the test set
print(full[0].shape, full[1].shape)
full_dataset_test = image_Dataset(image_paths=full[0],target_paths=full[1], use_flip = False)
MRI_test_loader   = DataLoader(full_dataset_test, batch_size= 4, shuffle=False, num_workers=0)
model.eval()
for i, (input, target, vis) in enumerate(MRI_test_loader):
    input, target = input.to(device), target.to(device, non_blocking=True)
    output = model(input) 
    output = output[-1]
    save_epoch_res_as_image2(input, output, target, epoch_num=i+1, target_th=0.5, pretext= True)
    extract_skeleton(input, output, target)
     

load image
retrieving ground truth coordinates
(39, 320, 320, 1) (39, 320, 320, 1)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3035: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/data/datafast1/DNS/newcode/src/train_utils.py:320: RuntimeWarning: invalid value encountered in true_divide
  ych = ych/np.max(np.max(ych))
/data/datafast1/DNS/newcode/src/train_utils.py:321: RuntimeWarning: invalid value encountered in less
  ych[np.where(ych<target_th)] = 0


(518, 1034, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less
  


(518, 1034, 3)
(518, 1034, 3)
(518, 1034, 3)
(518, 1034, 3)
(518, 1034, 3)
(518, 1034, 3)
(518, 1034, 3)
(518, 1034, 3)
(518, 1034, 3)


In [8]:
np.int(53.55555)

53